In [1]:
import os 
import numpy as np 
import pandas as pd 
from osgeo import gdal, osr
from tqdm.notebook import tqdm 
import requests 
import matplotlib.pyplot as plt 
from io import BytesIO

In [2]:
BASE_DIR = '.'
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data\\Nightlights\\2013\\F182013.v4c_web.stable_lights.avg_vis.tif')
COUNTRY = 'malawi_2016'
RANDOM_SEED = 7
PROCESSED_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY, 'processed')
IMAGE_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY, 'images')

In [3]:
os.makedirs(IMAGE_DIR, exist_ok=False)

# Generate candidate location

In [4]:
def latLon2Pixel(lat, lon, ct, gt):
    (lon, lat, holder) = ct.TransformPoint(lon, lat)
    x = (lon-gt[0])/gt[1]
    y = (lat-gt[3])/gt[5]
    return (int(x), int(y))


def locsToPixels(srcAddr, latLonPairs):
    ds = gdal.Open(srcAddr)
    gt = ds.GetGeoTransform()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(ds.GetProjection())
    srsLatLong = srs.CloneGeogCS()
    ct = osr.CoordinateTransformation(srsLatLong, srs)
    pixelPairs = []
    for point in latLonPairs:
        lat, lon = point
        pixelPairs.append(latLon2Pixel(lat, lon, ct, gt))
    return pixelPairs


def pixel2coord(x, y, xoff, a, b, yoff, d, e):
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return (yp, xp)

def pixelsToCoords(srcAddr, pixelPairs):
    ds = gdal.Open(srcAddr)
    xoff, a, b, yoff, d, e = ds.GetGeoTransform()
    latLonPairs = []
    for pixel in pixelPairs:
        x, y = pixel
        latLonPairs.append(pixel2coord(x + 0.5, y + 0.5, xoff, a, b, yoff, d, e))
    return latLonPairs


def get_download_locations():
    df = pd.read_csv(os.path.join(PROCESSED_DIR, 'clusters.csv'))
    clusters = df[['lat', 'lon']].values.tolist()
    top_lefts = [(lat + 0.045, lon - 0.045) for lat, lon in clusters]
    bottom_rights = [(lat - 0.045, lon + 0.045) for lat, lon in clusters]
    top_left_pixellocs = locsToPixels(NIGHTLIGHTS_DIR, top_lefts)
    bottom_right_pixellocs = locsToPixels(NIGHTLIGHTS_DIR, bottom_rights)
    output_cluster_locs = []
    output_pix = []
    for i in range(len(clusters)):
        top_left = top_left_pixellocs[i]
        bottom_right = bottom_right_pixellocs[i]
        for x in range(top_left[0], bottom_right[0]):
            for y in range(top_left[1], bottom_right[1]):
                output_pix.append((x, y))
                output_cluster_locs.append(clusters[i])

    output_locs = pixelsToCoords(NIGHTLIGHTS_DIR, output_pix)
    save_dir = os.path.join(PROCESSED_DIR, 'candidate_download_locs.csv')
    columns = ['im_lat', 'im_lon', 'cluster_lat', 'cluster_lon']
    with open(save_dir, 'w') as f:
        f.write(','.join(columns) + '\n')
        for loc, cluster_loc in zip(output_locs, output_cluster_locs):
            to_write = ','.join([str(loc[0]), str(loc[1]), str(cluster_loc[0]), str(cluster_loc[1])]) + '\n'
            f.write(to_write)
        print(f'{len(output_locs)} locations saved at {save_dir}')

In [5]:
print(f"Generating candidate image locations for {COUNTRY}")
#get_download_locations()

df= pd.read_csv(os.path.join(PROCESSED_DIR, 'clusters.csv'))
cluster

Generating candidate image locations for malawi_2016
